Normalización del dataset combinando los aspectos mencionados en las especificaciones

In [1]:
# Añadir el directorio 'src' al sys.path
import sys
import os

In [2]:
# Obtener la ruta absoluta de la carpeta 'src' para que sea accesible desde el notebook
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [3]:
# Importar la función de normalización
from preprocessing import normalize_text

In [4]:
# Importar pandas para cargar y manipular el dataset
import pandas as pd

In [5]:
# Cargar el dataset
df = pd.read_csv("../data/TA1C_dataset_detection_train.csv")


In [6]:
# Aplicar cada tipo de normalización
df['tokenized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="tokenization"))
df['cleaned_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="text_cleaning"))
df['no_stopwords_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="remove_stopwords"))
df['lemmatized_text'] = df['Teaser Text'].apply(lambda x: normalize_text(x, mode="lemmatization"))


In [7]:
# Mostrar ejemplos de cada tipo de normalización
print("\nEjemplos de normalización:")
print(df[['Teaser Text', 'tokenized_text', 'cleaned_text', 'no_stopwords_text', 'lemmatized_text']].head())



Ejemplos de normalización:
                                         Teaser Text  \
0  #SegundaDivisión  | La fortaleza del ataque: R...   
1  Jorge Lanata a los argentinos que se van a Uru...   
2  Raffo: “Los montevideanos deben estar alerta p...   
3  Ecos del universo: joven uruguayo desentraña (...   
4  Propuesta quinquenal de ANEP: aumento de 3,8% ...   

                                      tokenized_text  \
0  [#, SegundaDivisión,  , |, La, fortaleza, del,...   
1  [Jorge, Lanata, a, los, argentinos, que, se, v...   
2  [Raffo, :, “, Los, montevideanos, deben, estar...   
3  [Ecos, del, universo, :, joven, uruguayo, dese...   
4  [Propuesta, quinquenal, de, ANEP, :, aumento, ...   

                                        cleaned_text  \
0  segundadivisión   la fortaleza del ataque ramp...   
1  jorge lanata a los argentinos que se van a uru...   
2  raffo “los montevideanos deben estar alerta po...   
3  ecos del universo joven uruguayo desentraña y ...   
4  propuesta quinq

In [8]:

# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'


Dividimos el dataset dentro del Notebook

In [9]:
# Guardar el dataset con todas las columnas de normalización
output_path = "../data/TA1C_dataset_detection_train_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"Dataset con normalizaciones guardado en '{output_path}'")

Dataset con normalizaciones guardado en '../data/TA1C_dataset_detection_train_cleaned.csv'


In [12]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en conjuntos de entrenamiento y desarrollo
train_df, dev_df = train_test_split(
    df,
    test_size=0.25,  # 25% para desarrollo
    random_state=0,  # Para reproducibilidad
    stratify=df['Tag Value']  # Estratificar según la columna de clase
)

# Guardar los conjuntos en archivos CSV
train_df.to_csv("../data/TA1C_dataset_detection_train_split.csv", index=False)
dev_df.to_csv("../data/TA1C_dataset_detection_dev_split.csv", index=False)

print("Conjuntos de entrenamiento y desarrollo guardados en '../data/'")

Conjuntos de entrenamiento y desarrollo guardados en '../data/'


Representación de texto

In [13]:
# Cargar los conjuntos de entrenamiento y desarrollo
train_df = pd.read_csv("../data/TA1C_dataset_detection_train_split.csv")
dev_df = pd.read_csv("../data/TA1C_dataset_detection_dev_split.csv")

# Verificar las columnas disponibles
print(train_df.columns)

Index(['Tweet ID', 'Tweet Date', 'Media Name', 'Media Origin', 'Teaser Text',
       'Tag Value', 'tokenized_text', 'cleaned_text', 'no_stopwords_text',
       'lemmatized_text'],
      dtype='object')


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Probar diferentes configuraciones de n-gramas y representaciones
configurations = [
    {"ngram_range": (1, 1), "representation": "tfidf"},  # Unigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "tfidf"},  # Unigramas + Bigramas con TF-IDF
    {"ngram_range": (1, 3), "representation": "tfidf"},  # Unigramas + Bigramas + Trigramas con TF-IDF
    {"ngram_range": (1, 2), "representation": "binary"},  # Unigramas + Bigramas con representación binaria
    {"ngram_range": (1, 2), "representation": "frequency"},  # Unigramas + Bigramas con frecuencia
]

for config in configurations:
    print(f"Configuración: ngram_range={config['ngram_range']}, representación={config['representation']}")
    
    # Configurar el vectorizador
    if config["representation"] == "tfidf":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"])
    elif config["representation"] == "binary":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], binary=True)
    elif config["representation"] == "frequency":
        vectorizer = TfidfVectorizer(ngram_range=config["ngram_range"], use_idf=False)
    
    # Crear representaciones para el conjunto de entrenamiento y desarrollo
    X_train = vectorizer.fit_transform(train_df['lemmatized_text'])
    X_dev = vectorizer.transform(dev_df['lemmatized_text'])
    
    # Verificar las dimensiones
    print(f"Dimensiones de X_train: {X_train.shape}")
    print(f"Dimensiones de X_dev: {X_dev.shape}")

Configuración: ngram_range=(1, 1), representación=tfidf
Dimensiones de X_train: (2100, 9183)
Dimensiones de X_dev: (700, 9183)
Configuración: ngram_range=(1, 2), representación=tfidf
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)
Configuración: ngram_range=(1, 3), representación=tfidf
Dimensiones de X_train: (2100, 79900)
Dimensiones de X_dev: (700, 79900)
Configuración: ngram_range=(1, 2), representación=binary
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)
Configuración: ngram_range=(1, 2), representación=frequency
Dimensiones de X_train: (2100, 39506)
Dimensiones de X_dev: (700, 39506)


In [15]:
from sklearn.decomposition import TruncatedSVD

# Reducir dimensionalidad con TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0)  # Reducir a 100 dimensiones
X_train_reduced = svd.fit_transform(X_train)
X_dev_reduced = svd.transform(X_dev)

# Verificar las dimensiones después de la reducción
print(f"Dimensiones reducidas de X_train: {X_train_reduced.shape}")
print(f"Dimensiones reducidas de X_dev: {X_dev_reduced.shape}")

Dimensiones reducidas de X_train: (2100, 100)
Dimensiones reducidas de X_dev: (700, 100)


In [17]:
# Convertir etiquetas de texto a valores numéricos
label_mapping = {'Clickbait': 1, 'No': 0}
train_df['Tag Value'] = train_df['Tag Value'].map(label_mapping)
dev_df['Tag Value'] = dev_df['Tag Value'].map(label_mapping)

Validación Cruada

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

# Configurar validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Definir el modelo
model = LogisticRegression(max_iter=200)

# Evaluar el modelo con validación cruzada usando f1_macro
f1_macro_scorer = make_scorer(f1_score, average='macro')
scores = cross_val_score(model, X_train_reduced, train_df['Tag Value'], cv=cv, scoring=f1_macro_scorer)

# Imprimir los resultados
print(f"F1-macro scores por fold: {scores}")
print(f"F1-macro promedio: {scores.mean():.4f}")

F1-macro scores por fold: [0.57350414 0.57255747 0.61648997 0.60008246 0.55428571]
F1-macro promedio: 0.5834
